In [1]:
import pandas as pd
database = pd.read_csv('./data/limti.csv')


In [3]:
import spacy
nlp = spacy.load("en_core_web_sm")

/home/cqliu/anaconda3/envs/Conate/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from pdb import set_trace as stop

In [7]:
doc = nlp('I')
doc

I

In [9]:
doc[0].pos_

'PRON'

In [19]:
from tqdm import tqdm
import json
datasets = ['res14', 'res15', 'res16', 'lap14']
types = ['dev', 'test', 'train', ]
relation_need = ["RelatedTo", "FormOf", "IsA", "PartOf", "HasA","UsedFor", 'Synonym', 'SimilarTo']
relation2id = {"RelatedTo":0, "FormOf":1, "IsA":2, "PartOf":3, "HasA":4,"UsedFor":5, 'Synonym':6, 'SimilarTo':7, 'notIn':8}
from pdb import set_trace as stop
for dataset in datasets:
    for type in types:
        with open('./data/{0}/{1}/sentence.txt'.format(dataset, type), 'r') as f:
            sent = f.readlines()
        with open('./data/{0}/{1}/target.txt'.format(dataset, type), 'r') as f:
            target = f.readlines()
        Allresult = []
        for sen, tar in tqdm(zip(sent, target), ncols=80, desc='Handling Data'):
            singresult = []
            newsen = sen.strip()
            newtar = tar.strip()
            assert len(newsen.split()) == len(newtar.split())
            # noun_chunck = list(i for i in nlp(newsen).noun_chunks)
            doc = nlp(newsen)
            noun_chunck = [chunk.text.replace(chunk[0].text + " ", "") if chunk[0].pos_ == "DET" else chunk.text for chunk in doc.noun_chunks]
            chunck_dict_relation = {}
            chunck_dict_id = {}
            judge = []
            singCom = []
            singId = []
            for i in noun_chunck:
                judge.extend(str(i).split())
            for chunck in noun_chunck:
                chunck = str(chunck)
                if len(chunck.split()) == 1 and nlp(chunck)[0].pos_ == 'PRON' or len(chunck) == 1 or chunck == 'people':
                    chunck_dict_relation[chunck] = list()
                    chunck_dict_id[chunck] = list()
                else:
                    chunck_dict_relation[chunck] = list(database[database['start_node']==chunck].groupby('relation').head(6)['end_node'])
                    chunck_dict_id[chunck] = [relation2id[i] for i in list(database[database['start_node']==chunck].groupby('relation').head(6)['relation'])]
            for word in newsen.split():
                if word in judge:
                    for key in chunck_dict_relation.keys():
                        # the 存在与they中
                        if word in key:
                            com = chunck_dict_relation[key]
                            comid = chunck_dict_id[key]
                            if len(com) == 0:
                                singCom.append(word+' '+str(key).replace(' ', '_'))
                                singId.append('-1 '+str(relation2id['notIn']))
                            else:
                                singCom.append(word+' '+' '.join([str(i) for i in com]))
                                singId.append('-1'+' '+' '.join([str(i) for i in comid]))
                            break
                else:
                    singCom.append(word)
                    singId.append('-1')
            singresult.append(newsen)
            singresult.append(newtar)
            singresult.append(singCom)
            singresult.append(singId)
            Allresult.append(singresult)
        with open('./newData/{0}{1}.json'.format(dataset, type), 'w') as f:
            json.dump(Allresult, f)

Handling Data: 150it [01:53,  1.32it/s]
Handling Data: 800it [11:32,  1.15it/s]
Handling Data: 1231it [15:52,  1.09s/it]